### Imports

In [12]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [ ]:

# Load data files
distanceDF = pd.read_csv("data/Distance.csv", header=None)   # Distance between counties
capacityDF = pd.read_csv("data/Capacity.csv", header=None)   # Healthcare facility capacities
populationDF = pd.read_csv("data/Population.csv", header=None)  # Population per county
vaccineDF = pd.read_csv("data/Florida_Weekly_Vaccine_Summary.csv")  # Weekly vaccination data
vaccination_rates = vaccineDF['vaccination_rate'].tolist()  # List of weekly vaccination rates

# Define model parameters
t_B = 0                # Start time period
t_E = 26               # End time period (26 weeks)
psi = 1                # Decision interval
numCounties = 67       # Number of counties in Florida

# Define sets for counties, healthcare facilities, time periods, and decision periods
I = range(1, numCounties + 1)         # Set of counties
J = range(1, numCounties + 1)         # Set of healthcare facilities (assuming same as counties for simplicity)
T = range(t_B, t_E + 1)               # Set of time periods
T_prime = range(t_B, t_E + 1, psi)    # Set of decision-making time periods

# Initial values for SIRV model
initial_S = [0.99 for _ in range(numCounties + 1)]  # Initial susceptible fraction
initial_I = [0.01 for _ in range(numCounties + 1)]  # Initial infected fraction
initial_R = [0.00 for _ in range(numCounties + 1)]  # Initial recovered fraction
initial_X = [0.00 for _ in range(numCounties + 1)]  # Initial vaccinated fraction

# Model parameters with example values
beta = {i: 0.1 for i in I}            # Transmission rate for each county
v = {i: 0.05 for i in I}              # Recovery rate for each county
l = {i: 0.05 for i in I}              # Leaky vaccine probability for each county
rho = {i: 0.1 for i in I}             # Hospitalization rate for each county
C = {j: capacityDF.iloc[j-1, 0] for j in J}  # Capacity at each healthcare facility
V_total = {t: 1000 for t in T_prime}  # Total vaccines available at each decision period
d = {(i, j): distanceDF.iloc[i-1, j-1] for i in I for j in J}  # Distance between counties
D_max = 75                            # Max travel distance allowed for healthcare
N = {i: populationDF.iloc[i-1, 0] for i in I}  # Population in each county
M = 1e6                               # Large number for big-M constraints in linear programming
z = {(i, t): vaccination_rates[t] for i in I for t in T}    # Assign rate for each county and time period

# Initialize Gurobi model
model = gp.Model("SIRV_Model")

# Define decision variables
S = model.addVars(I, T, lb=0, name="S")             # Susceptible population per county and time
I_var = model.addVars(I, T, lb=0, name="I")         # Infected population per county and time
R = model.addVars(I, T, lb=0, name="R")             # Recovered population per county and time
X = model.addVars(I, T, lb=0, name="X")             # Vaccinated population per county and time
H = model.addVars(I, T, lb=0, name="H")             # Hospitalized population per county and time
V = model.addVars(I, T_prime, lb=0, name="V")       # Vaccines allocated per county and decision period
y = model.addVars(I, J, T_prime, lb=0, name="y")    # Number of people traveling for healthcare per county, facility, and time
u = model.addVars(I, T_prime, lb=0, name="u")       # Unmet healthcare demand per county and time
b = model.addVars(I, J, T_prime, vtype=GRB.BINARY, name="b")  # Binary travel variable
W_SI = model.addVars(I, T, lb=0, name="W_SI")       # Auxiliary variable for linearizing S*I
Z_XI = model.addVars(I, T, lb=0, name="Z_XI")       # Auxiliary variable for linearizing X*I

# Objective: Minimize total unmet healthcare demand
model.setObjective(gp.quicksum(u[i, t] for i in I for t in T_prime), GRB.MINIMIZE)

# Constraints
for i in I:
    for t in T:
        if t == t_B:
            # Set initial values for S, I, R, X at the start time
            model.addConstr(S[i, t] == initial_S[i])
            model.addConstr(I_var[i, t] == initial_I[i])
            model.addConstr(R[i, t] == initial_R[i])
            model.addConstr(X[i, t] == initial_X[i])
        else:
            # Update equations for S, I, R, X based on SIRV model dynamics
            model.addConstr(S[i, t] == S[i, t-1] - beta[i] * W_SI[i, t-1] - z[i, t] * S[i, t-1])
            model.addConstr(I_var[i, t] == I_var[i, t-1] + beta[i] * (W_SI[i, t-1] + l[i] * Z_XI[i, t-1]) - v[i] * I_var[i, t-1])
            model.addConstr(R[i, t] == R[i, t-1] + v[i] * I_var[i, t-1])
            model.addConstr(X[i, t] == X[i, t-1] + z[i, t] * S[i, t-1] - l[i] * beta[i] * Z_XI[i, t-1])

        # Constraint for hospitalization demand
        model.addConstr(H[i, t] == rho[i] * I_var[i, t])

# Unmet demand constraint
for i in I:
    for t in T_prime:
        model.addConstr(u[i, t] == H[i, t] - gp.quicksum(y[i, j, t] for j in J))

# Healthcare facility capacity constraint
for j in J:
    for t in T_prime:
        model.addConstr(gp.quicksum(y[i, j, t] for i in I) <= C[j])

# Vaccine allocation constraints
for t in T_prime:
    model.addConstr(gp.quicksum(V[i, t] for i in I) <= V_total[t])

for i in I:
    for t in T_prime:
        model.addConstr(z[i, t] * S[i, t] <= V[i, t])

# Travel constraints for access to healthcare facilities
for i in I:
    for j in J:
        for t in T_prime:
            model.addConstr(y[i, j, t] <= M * b[i, j, t])
            model.addConstr(d[i, j] * b[i, j, t] <= D_max)
            model.addConstr(y[i, j, t] >= b[i, j, t])

# Linearization constraints for W_SI and Z_XI
for i in I:
    for t in T:
        model.addConstr(W_SI[i, t] >= N[i] * I_var[i, t] + S[i, t] * N[i] - N[i] ** 2)
        model.addConstr(W_SI[i, t] <= N[i] * I_var[i, t])
        model.addConstr(W_SI[i, t] <= N[i] * S[i, t])

        model.addConstr(Z_XI[i, t] >= N[i] * I_var[i, t] + X[i, t] * N[i] - N[i] ** 2)
        model.addConstr(Z_XI[i, t] <= N[i] * I_var[i, t])
        model.addConstr(Z_XI[i, t] <= N[i] * X[i, t])

# Optimize the model
model.optimize()

# Output results if optimal solution is found
if model.status == GRB.OPTIMAL:
    for i in I:
        for t in T:
            print(f"County {i}, Time {t} - Susceptible: {S[i, t].x}, Infected: {I_var[i, t].x}, Recovered: {R[i, t].x}, Vaccinated: {X[i, t].x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 388962 rows, 258687 columns and 907030 nonzeros
Model fingerprint: 0x5ff1a837
Variable types: 137484 continuous, 121203 integer (121203 binary)
Coefficient statistics:
  Matrix range     [5e-05, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 4e+06]
Presolve removed 362623 rows and 236041 columns
Presolve time: 0.12s
Presolved: 26339 rows, 22646 columns, 67904 nonzeros
Variable types: 15960 continuous, 6686 integer (6685 binary)
Found heuristic solution: objective 10.0453892

Root relaxation: objective 8.006950e+00, 2281 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb